In [4]:
import pandas as pd
import numpy as np

df = pd.read_csv("data\messy_data.csv")
df

,carat,clarity,color,cut,x dimension,y dimension,z dimension,depth,table,price
0,0.50,IF,D,Ideal,5.1,5.15,3.2,61.5,,3000
1,0.70,vvs2,E,premium,5.7,,3.52,62,59,4500
2,NaN,Si2,h,Good,4.3,4.31,,62.3,56,700
3,1.20,if,d,ideal,,6.82,4.2,61.7,58,10000
4,0.90,I1,J,Fair,6,,3.7,61.7,,2400
...,...,...,...,...,...,...,...,...,...,...
195,0.45,si1,F,very good,4.9,4.89,,62.4,59,2300
196,1.25,VVS2,H,Good,6.9,,4.27,,55,10400
197,0.55,i1,I,Premium,5.3,,3.28,62.3,57,150000
198,0.85,Si2,E,fair,6.1,6.13,3.78,,58,6300


remove all whitespace characters from columns names, and change x,y,z dimentions whitespaces to '_'

In [5]:
df.columns=df.columns.str.replace(' ','')
df.rename(columns={
    'xdimension':'x_dimension',
    'ydimension':'y_dimension',
    'zdimension':'z_dimension',
},inplace=True)

check how many fields are nan

In [6]:
df.isna().sum()

carat          50
clarity         0
color           0
cut             0
x_dimension     0
y_dimension     0
z_dimension     0
depth           0
table           0
price           0
dtype: int64

lets replace empty strings with NaNs

In [7]:
df=df.replace(r'^\s*$', np.nan, regex=True)
df.isna().sum()


carat          50
clarity         0
color           0
cut             0
x_dimension     3
y_dimension    64
z_dimension    48
depth          67
table          33
price           4
dtype: int64

lets check columns types

In [8]:
df.dtypes

carat          float64
clarity         object
color           object
cut             object
x_dimension     object
y_dimension     object
z_dimension     object
depth           object
table           object
price           object
dtype: object

that`s not right, lets fix it

In [9]:
df['clarity']=df['clarity'].astype('category')
df['color']=df['color'].astype('category')
df['cut']=df['cut'].astype('category')
df['x_dimension']=df['x_dimension'].astype('float64')
df['y_dimension']=df['y_dimension'].astype('float64')
df['z_dimension']=df['z_dimension'].astype('float64')
df['depth']=df['depth'].astype('float64')
df['table']=df['table'].astype('float64')
df['price']=df['price'].astype('float64')

seems legit. now lets remove all duplicates

In [10]:
pre_duplicate_removal_size =len(df)
df.drop_duplicates()
print(f'removed {len(df)-pre_duplicate_removal_size} duplicated rows')

removed 0 duplicated rows


no duplicates, that good I suppose. now lets fil those NaNs

In [11]:
df.isna().sum()

carat          50
clarity         0
color           0
cut             0
x_dimension     3
y_dimension    64
z_dimension    48
depth          67
table          33
price           4
dtype: int64

In [12]:
#those two can be mean, overall size of each is less than 5% of data frame
df['price'].fillna(df['price'].mean(), inplace=True)
df['x_dimension'].fillna(df['x_dimension'].mean(), inplace=True)

#lets fil those with median
df['y_dimension'].fillna(df['y_dimension'].median(), inplace=True)
df['z_dimension'].fillna(df['z_dimension'].median(), inplace=True)
df['depth'].fillna(df['depth'].median(), inplace=True)

#and carat with most frequent
df['carat'].fillna(df['carat'].mode().iloc[0], inplace=True)
df['table'].fillna(df['table'].mode().iloc[0], inplace=True)
df.isna().sum()

carat          0
clarity        0
color          0
cut            0
x_dimension    0
y_dimension    0
z_dimension    0
depth          0
table          0
price          0
dtype: int64

move all category names to upper, thus removing human typing errors

In [13]:
df['color']=df['color'].str.upper()
df['cut']=df['cut'].str.upper()
df['clarity']=df['clarity'].str.upper()
df

,carat,clarity,color,cut,x_dimension,y_dimension,z_dimension,depth,table,price
0,0.50,IF,D,IDEAL,5.100000,5.15,3.200,61.5,56.0,3000.0
1,0.70,VVS2,E,PREMIUM,5.700000,5.85,3.520,62.0,59.0,4500.0
2,0.50,SI2,H,GOOD,4.300000,4.31,3.715,62.3,56.0,700.0
3,1.20,IF,D,IDEAL,5.885787,6.82,4.200,61.7,58.0,10000.0
4,0.90,I1,J,FAIR,6.000000,5.85,3.700,61.7,56.0,2400.0
...,...,...,...,...,...,...,...,...,...,...
195,0.45,SI1,F,VERY GOOD,4.900000,4.89,3.715,62.4,59.0,2300.0
196,1.25,VVS2,H,GOOD,6.900000,5.85,4.270,62.3,55.0,10400.0
197,0.55,I1,I,PREMIUM,5.300000,5.85,3.280,62.3,57.0,150000.0
198,0.85,SI2,E,FAIR,6.100000,6.13,3.780,62.3,58.0,6300.0


now lets check for outliers

In [14]:
df['carat'].describe()

count    200.00000
mean       0.72425
std        0.30724
min        0.20000
25%        0.50000
50%        0.57500
75%        0.95000
max        1.60000
Name: carat, dtype: float64

In [15]:
df['x_dimension'].describe()

count    200.000000
mean       5.885787
std        0.764344
min        3.900000
25%        5.300000
50%        5.900000
75%        6.500000
max        7.300000
Name: x_dimension, dtype: float64

In [16]:
df['y_dimension'].describe()

count    200.000000
mean       5.836200
std        0.632138
min        3.890000
25%        5.580000
50%        5.850000
75%        6.165000
max        7.280000
Name: y_dimension, dtype: float64

In [17]:
df['z_dimension'].describe()

count    200.000000
mean       3.699550
std        0.402912
min        2.410000
25%        3.477500
50%        3.715000
75%        3.970000
max        4.500000
Name: z_dimension, dtype: float64

In [18]:
df['price'].describe()

count       200.000000
mean      11410.714286
std       39667.884273
min         500.000000
25%        2600.000000
50%        4450.000000
75%        8025.000000
max      350000.000000
Name: price, dtype: float64

price and carat should be cleared of outliers, especially price. lets check if price quantile range of 1 to 99 will do the job 

now for carat, min seems good enought, lets filter max 

In [20]:
df['carat'].describe()

count    186.000000
mean       0.733602
std        0.300742
min        0.350000
25%        0.500000
50%        0.600000
75%        0.950000
max        1.600000
Name: carat, dtype: float64

In [21]:
high = df['carat'].quantile(0.99)
print(f'quantiles 99 = {high}')
print('nope lets go for 95')
high = df['carat'].quantile(0.95)
print(f'quantiles 95 = {high}')
df=df[(df['carat']<=high )]
df['carat'].describe()

quantiles 99 = 1.4150000000000005
nope lets go for 95
quantiles 95 = 1.3


count    179.000000
mean       0.706425
std        0.272010
min        0.350000
25%        0.500000
50%        0.600000
75%        0.900000
max        1.300000
Name: carat, dtype: float64

how many records do we have now?

In [22]:
len(df)

179

lets stay there, difference is smaller that in original data set